In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
import torch.nn as nn
import torch

#Leer DataSet

X = pd.read_csv('DataSet_Celestial_Body.csv')
y = pd.read_csv('Outputs_Celestial_Body.csv')

X = X.iloc[:,1]
y = y.iloc[:,1]

#Leer DataSet_val

X_val = pd.read_csv('DataSet_Celestial_Body_val.csv')
y_val = pd.read_csv('Outputs_Celestial_Body_val.csv')

X_val = X_val.iloc[:,1]
y_val = y_val.iloc[:,1]

#Leer DataSet_test

X_test = pd.read_csv('DataSet_Celestial_Body_test.csv')
y_test = pd.read_csv('Outputs_Celestial_Body_test.csv')

X_test = X_test.iloc[:,1]
y_test = y_test.iloc[:,1]

#Imprimir :V
#print("Dataset - Validationset - Testset")
#print(X)
#print(y)
#print(X_val)
#print(y_val)
#print(X_test)
#print(y_test)

#Convertir a un numpy array
X = np.array(X)
y = np.array(y)

X_val = np.array(X_val)
y_val = np.array(y_val)

X_test = np.array(X_test)
y_test = np.array(y_test)

#Reshape a los Dataset
X = X.reshape((2000*9,3))
y = y.reshape((2000*9,9))

X_val = X_val.reshape((650*9,3))
y_val = y_val.reshape((650*9,9))

X_test = X_test.reshape((650*9,3))
y_test = y_test.reshape((650*9,9))

#Imprimir :V
print("Dataset y Output")
print(X)
print(y)
print("Validationset y ValidationOutput")
print(X_val)
print(y_val)
print("Testset y TestOutput")
print(X_test)
print(y_test)

Dataset y Output
[[-0.37607108  2.85955243 -0.59824457]
 [-0.37607108  2.80196128 -0.59968969]
 [-0.37607108  2.83104483 -0.60010501]
 ...
 [-0.37607139 -0.35391866 -0.50343915]
 [-0.37607139 -0.35391679 -0.49875621]
 [-0.37607139 -0.35391748 -0.49862577]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Validationset y ValidationOutput
[[-0.37607085  2.82597894 -0.60322265]
 [-0.37607085  2.83145461 -0.59768046]
 [-0.37607086  2.85639056 -0.58529868]
 ...
 [-0.37607117 -0.353915   -0.48879441]
 [-0.37607117 -0.35391892 -0.50745619]
 [-0.37607117 -0.35391787 -0.50123163]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Testset y TestOutput
[[-0.37607144  2.79877045 -0.60262377]
 [-0.37607144  2.82808602 -0.59722985]
 [-0.37607144  2.84411679 -0.59752437]
 ...
 [-0.37607176 -

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 18)
        self.g1 = nn.ReLU()
        self.fc2 = nn.Linear(18, 18)
        self.g2 = nn.ReLU()
        self.fc3 = nn.Linear(18, 9)
        self.g3 = nn.Softmax()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.g1(out)
        out = self.fc2(out)
        out = self.g2(out)
        out = self.fc3(out)
        out = self.g3(out)
        
        return out

In [8]:
net = Net()
criterio = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(net.parameters(), lr=0.1)

In [9]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
            
        yield inputs[excerpt], targets[excerpt]
        
for epoch in range(7):
    for x_batch,y_batch in iterate_minibatches(X,np.argmax(y,axis=-1),batchsize=32,shuffle=True):
        optimizer.zero_grad()
        y_hat = net(torch.from_numpy(x_batch).float())
        
        loss = criterio(y_hat, torch.from_numpy(y_batch).long())
        
        loss.backward()
        optimizer.step()
        
    print(loss.data[0])

100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:03<00:00, 183.86it/s]
C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(1.9919)


100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:02<00:00, 260.33it/s]


tensor(1.7407)


100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:01<00:00, 287.50it/s]


...

tensor(1.5933)


100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:01<00:00, 296.19it/s]


tensor(1.6231)


100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:01<00:00, 299.15it/s]


tensor(1.6353)


In [11]:
y_hat = net(torch.from_numpy(X_test).float())
preds = y_hat.detach().numpy()
print(y_hat.shape)

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([5850, 9])


In [13]:
print("Acuracy en TestSet")
accuracy_score(np.argmax(preds, axis=-1), np.argmax(y_test, axis=-1))
print("Matris de Confucion en TestSet")
confusion_matrix(np.argmax(preds, axis=-1), np.argmax(y_test, axis=-1))

Acuracy en TestSet
Matris de Confucion en TestSet


array([[650,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 650,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 650,   0, 650,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 650,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 650,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 650,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 650,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 650]], dtype=int64)